<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Libraries</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Preprocessing</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Feature Engineering</a></span></li><li><span><a href="#Train-&amp;-Test" data-toc-modified-id="Train-&amp;-Test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Train &amp; Test</a></span><ul class="toc-item"><li><span><a href="#Train,-Test,-Split" data-toc-modified-id="Train,-Test,-Split-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Train, Test, Split</a></span></li><li><span><a href="#Check-train/test-shape" data-toc-modified-id="Check-train/test-shape-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Check train/test shape</a></span></li></ul></li></ul></div>

# Import Libraries

In [3]:
# Import all packages that I might need:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.model_selection import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score
from scipy import stats

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

# Preprocessing

In [5]:
# Create dataframe from zillow median home price data
df = pd.read_csv(r'/Users/jimpetoskey/Documents/Flatiron/phase5/data/stock_prices.csv')

# make copy of df
df1 = df.copy()

# Look at top-performing stocks, choose top-performer to model
df1.sort_values(by='Target', ascending=False)

df.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [9]:
# Pull one stock's values for given stock from stock_prices
values8103 = df1.loc[df1['SecuritiesCode'] == 8103]                   
values8103.head(2)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
1450,20170104_8103,2017-01-04,8103,406.0,413.0,395.0,413.0,82600,1.0,NaN,False,-0.024213
3315,20170105_8103,2017-01-05,8103,415.0,419.0,408.0,413.0,57500,1.0,NaN,False,-0.002481


# Feature Engineering

In [4]:
def feature_eng(train):
    # Bool to int for SupervisionFlag
    train["SupervisionFlag"] = train["SupervisionFlag"].astype(int)
    
    # Forward, then backward fill missing values in cols
    cols = ['Open', 'High', 'Low', 'Close', 'Target']
    train.loc[:,cols] = train.loc[:,cols].ffill()
    train.loc[:,cols] = train.loc[:,cols].bfill()
    
    # Replace missing values in ExpectedDividend with 0
    train['ExpectedDividend'] = train['ExpectedDividend'].fillna(0)
    
    # Add Features
    train['Daily_Spread'] = train['Close'] - train['Open']
    train['1_Day_Spread'] = train['Close'].diff()
    train['2_Day_Spread'] = train['Close'].diff(periods=2)
    train['1_Week_Spread'] = train['Close'].diff(periods=5)
    
    # Fill missing values with 0
    train = train.fillna(0)
    
    # Add zscored columns and drop raw columns
    train['open'] = stats.zscore(train['Open'])
    train['high'] = stats.zscore(train['High'])
    train['low'] = stats.zscore(train['Low'])
    train['close'] = stats.zscore(train['Close'])
    train['volume'] = stats.zscore(train['Volume'])
    train['daily_spread'] = stats.zscore(train['Daily_Spread'])
    train['1_day_spread'] = stats.zscore(train['1_Day_Spread'])
    train['2_day_spread'] = stats.zscore(train['2_Day_Spread'])
    train['1_week_spread'] = stats.zscore(train['1_Week_Spread'])

    train = train.drop(['1_Day_Spread',
                        'Daily_Spread',
                        'Volume',
                        'Close',
                        'Low',
                        'High',
                        'Open',
                        '2_Day_Spread',
                        '1_Week_Spread'], axis=1)
    
    # Drop redundant id column
    train = train.drop(['RowId', 'Date'], axis=1)
    
    return train



In [14]:
five = feature_eng(values8103)

<ipython-input-4-cd303487fa13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["SupervisionFlag"] = train["SupervisionFlag"].astype(int)
/Users/jimpetoskey/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
<ipython-input-4-cd303487fa13>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [15]:
five

,SecuritiesCode,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,open,high,low,close,volume,daily_spread,1_day_spread,2_day_spread,1_week_spread
1450,8103,1.0,0.0,0,-0.024213,-0.725325,-0.698932,-0.779787,-0.667329,-0.149829,0.648330,-0.019387,-0.023000,-0.032175
3315,8103,1.0,0.0,0,-0.002481,-0.650578,-0.650293,-0.669449,-0.667329,-0.207087,-0.148674,-0.019387,-0.023000,-0.032175
5180,8103,1.0,0.0,0,-0.017413,-0.741935,-0.723252,-0.728862,-0.750732,-0.200015,-0.060118,-0.566415,-0.352508,-0.032175
7045,8103,1.0,0.0,0,0.007595,-0.741935,-0.739465,-0.754324,-0.759072,-0.242901,-0.148674,-0.074090,-0.385459,-0.032175
8910,8103,1.0,0.0,0,0.010050,-0.733630,-0.763785,-0.822224,-0.817454,-0.210052,-0.857122,-0.402307,-0.286606,-0.032175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324104,8103,1.0,0.0,0,0.008547,2.787782,2.786894,2.869852,2.793900,1.009467,-0.060118,-0.894632,-0.747918,-0.087110
2326104,8103,1.0,0.0,0,0.000000,2.870834,2.811214,2.810439,2.718837,0.980496,-1.742682,-0.511712,-0.846771,-0.471654
2328104,8103,1.0,0.0,0,0.015738,2.729646,2.705828,2.818927,2.777219,0.672308,0.382662,0.363532,-0.088901,-0.361784
2330104,8103,1.0,0.0,0,0.004768,2.779477,2.697722,2.810439,2.777219,0.591327,-0.148674,-0.019387,0.207656,-0.471654


In [13]:
values8103_copy = values8103.copy()

# Bool to int for SupervisionFlag
values8103_copy["SupervisionFlag"] = values8103_copy["SupervisionFlag"].astype(int)
    
# Forward, then backward fill missing values in cols
cols = ['Open', 'High', 'Low', 'Close', 'Target']
values8103_copy.loc[:,cols] = values8103_copy.loc[:,cols].ffill()
values8103_copy.loc[:,cols] = values8103_copy.loc[:,cols].bfill()

values8103_copy.isna().sum()

RowId                  0
Date                   0
SecuritiesCode         0
Open                   0
High                   0
Low                    0
Close                  0
Volume                 0
AdjustmentFactor       0
ExpectedDividend    1192
SupervisionFlag        0
Target                 0
dtype: int64

# Train & Test

## Train, Test, Split

In [8]:
# create X and y df's
X = values8103.drop('Target', axis=1)
y = values8103['Target']

In [9]:
# Run Train, Test, Split with 85% train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

## Check train/test shape

In [10]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', X_test.shape)
print('Testing Features Shape:', y_train.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (1021, 15)
Training Labels Shape: (181, 15)
Testing Features Shape: (1021,)
Testing Labels Shape: (181,)
